# Chapter 2. Redis Basics

## 2.1 Meet the Redis data types

Redis can store data as five different data types:

* Strings
* List
* Set
* Hashes
* Sorted set

## 2.2 Strings

(1) Don't have to be text of strings.

(2) Can be any kind of data, e.g., images, serialized objects, as long as <= 512MB.

(3) String basic commands:

* SET
* GET
* APPEND
* INCR and DECR
* GETRANGE
* MGET
* MSET
* STRLEN

### 2.2.1 String Get and Set

```redis
127.0.0.1:6379> set user "name: joe"
OK
127.0.0.1:6379> get user
"name: joe"
127.0.0.1:6379> set user:1 "{'name' : 'joe', 'email' : 'joe@joe.com'}"
OK
127.0.0.1:6379> get user:1
"{'name' : 'joe', 'email' : 'joe@joe.com'}"
```

### 2.2.2 Incrementing

```redis
127.0.0.1:6379> set user:id 1
OK
127.0.0.1:6379> get user:id
"1"
127.0.0.1:6379> incr user:id
(integer) 2
127.0.0.1:6379> get user:id
"2"
127.0.0.1:6379> append user:1 "extra data"
(integer) 51
127.0.0.1:6379> get user:1
"{'name' : 'joe', 'email' : 'joe@joe.com'}extra data"
```

### 2.2.3 Getrange

```redis
127.0.0.1:6379> set customer:1 "ABCDE00123"
OK
127.0.0.1:6379> get customer:1
"ABCDE00123"
127.0.0.1:6379> getrange customer:1 5 9
"00123"
```

### 2.2.4 Mget and Mset

Get/Set a batch of data.

* MGET = Multi-GET
* MSET = Multi-SET

```redis
127.0.0.1:6379> mset order:1 "order 1 data" order:2 "order 2 data"
OK
127.0.0.1:6379> mget order:1 order:2
1) "order 1 data"
2) "order 2 data"
127.0.0.1:6379> strlen user:1
(integer) 51
```

## 2.3 Lists

(1) A sorted list of strings, sorted by insertion order.

(2) New items can be inserted either at the begining or at the end of the list.

(3) List basic commands:

* LPUSH and RPUSH
* LREM
* LSET
* LINDEX
* LRANGE
* LLEN
* LPOP and RPOP
* LTRIM

### 2.3.1 Lpush and Rpush

```redis
127.0.0.1:6379> lpush recentcomments "Comment 1"
(integer) 1
127.0.0.1:6379> lrange recentcomments 0 1
1) "Comment 1"
127.0.0.1:6379> lrange recentcomments 0 0
1) "Comment 1"
127.0.0.1:6379> lpush recentcomments "Comment 2"
(integer) 2
127.0.0.1:6379> lpush recentcomments "Comment 3" "Comment 4"
(integer) 4
127.0.0.1:6379> lrange recentcomments 0 3
1) "Comment 4"
2) "Comment 3"
3) "Comment 2"
4) "Comment 1"
127.0.0.1:6379> rpush recentcomments "Comment 5"
(integer) 5
127.0.0.1:6379> rpush recentcomments "Comment 6"
(integer) 6
127.0.0.1:6379> lrange recentcomments 3 0
(empty list or set)
127.0.0.1:6379> lrange recentcomments 0 5
1) "Comment 4"
2) "Comment 3"
3) "Comment 2"
4) "Comment 1"
5) "Comment 5"
6) "Comment 6"
```

### 2.3.2 Trimming lists.

```redis
127.0.0.1:6379> ltrim recentcomments 0 4
OK
127.0.0.1:6379> lrange recentcomments 0 5
1) "Comment 4"
2) "Comment 3"
3) "Comment 2"
4) "Comment 1"
5) "Comment 5"
```

### 2.3.3 Other list commands

`LINDEX` can be used to get an item in the middle but it is not efficient in a huge linked list.

```redis
127.0.0.1:6379> lindex recentcomments 2
"Comment 2"
127.0.0.1:6379> lrange recentcomments 0 5
1) "Comment 4"
2) "Comment 3"
3) "Comment 2"
4) "Comment 1"
5) "Comment 5"
127.0.0.1:6379> lpop recentcomments
"Comment 4"
127.0.0.1:6379> lrange recentcomments 0 5
1) "Comment 3"
2) "Comment 2"
3) "Comment 1"
4) "Comment 5"
127.0.0.1:6379> lpop recentcomments
"Comment 3"
127.0.0.1:6379> lrange recentcomments 0 5
1) "Comment 2"
2) "Comment 1"
3) "Comment 5"
```

## 2.4 Sets

(1) A collection of unique strings.

(2) Adding, removing or testing for the existence of a member can be done in constant time.

(3) Sets don't keep the order of members as they are added.

(4) Set basic commands:

* SADD
* SCARD
* SDIFF, SINTER, and SUNION
* SISMEMBER
* SMEMBERS
* SMOVE
* SREM

### 2.4.1 Adding to sets

```redis
127.0.0.1:6379> sadd post:1:likes "joe" "bob" "mary"
(integer) 3
127.0.0.1:6379> scard post:1:likes
(integer) 3
127.0.0.1:6379> smembers post:1:likes
1) "joe"
2) "bob"
3) "mary"
127.0.0.1:6379> sadd post:2:likes "joe" "tim"
(integer) 2
127.0.0.1:6379> sdiff post:1:likes post:2:likes
1) "bob"
2) "mary"
127.0.0.1:6379> sinter post:1:likes post:2:likes
1) "joe"
127.0.0.1:6379> sunion post:1:likes post:2:likes
1) "joe"
2) "tim"
3) "bob"
4) "mary"
```

## 2.5 Hashes

(1) Maps between strings fields and string values.

(2) If a hash contains no more than 100 fields, it is stored in a very efficient way.

(3) Easier to get a particular field of the data, rather than getting the entire set of data.

(4) Hash basic commands:

* HSET
* HMSET
* HGET
* HMGET
* HGETALL
* HDEL
* HEXISTS
* HINCRBY
* HKEYS
* HVALS

```redis
127.0.0.1:6379> hset user:1:h name "joe"
(integer) 1
127.0.0.1:6379> hget user:1:h name
"joe"
127.0.0.1:6379> hmset user:1:h email "joe@joe.com" id 1
OK
127.0.0.1:6379> hmget user:1:h name email id
1) "joe"
2) "joe@joe.com"
3) "1"
```

### 2.5.1 Other hash commands

```redis
127.0.0.1:6379> hgetall user:1:h
1) "name"
2) "joe"
3) "email"
4) "joe@joe.com"
5) "id"
6) "1"
127.0.0.1:6379> hkeys user:1:h
1) "name"
2) "email"
3) "id"
127.0.0.1:6379> hvals user:1:h
1) "joe"
2) "joe@joe.com"
3) "1"
```

## 2.6 Sorted sets

(1) Sets that are sorted.

(2) Each member is associated with a score for sorting.

(3) Adding, removing, and updating items are very fast, so is getting a subset of items within a range of scores.

(4) Sorted set basic commands:

* ZADD
* ZCARD
* ZCOUNT
* ZINCRBY
* ZRANGE
* ZRANK
* ZREM
* ZSCORE

```redis
127.0.0.1:6379> zadd hs 120 "joe" 100 "bob" 150 "mary" 90 "tim"
(integer) 4
127.0.0.1:6379> zrange hs 0 4
1) "tim"
2) "bob"
3) "joe"
4) "mary"
127.0.0.1:6379> zrange hs 0 3 WITHSCORES
1) "tim"
2) "90"
3) "bob"
4) "100"
5) "joe"
6) "120"
7) "mary"
8) "150"
127.0.0.1:6379> zadd hs 125 "joe"
(integer) 0
127.0.0.1:6379> zrange hs 0 3 WITHSCORES
1) "tim"
2) "90"
3) "bob"
4) "100"
5) "joe"
6) "125"
7) "mary"
8) "150"
```

### 2.6.1 Other sorted set commands

```redis
127.0.0.1:6379> zrank hs bob
(integer) 1
127.0.0.1:6379> zrank hs tim
(integer) 0
127.0.0.1:6379> zscore hs tim
"90"
127.0.0.1:6379> zcard hs
(integer) 4
127.0.0.1:6379> zcount hs 90 120
(integer) 2
```

## 2.7 Pub and sub

Redis can be used as a message bus.

### 2.7.1 Using pub and sub

(1) Subscribe to a message channel.

```redis
127.0.0.1:6379> subscribe greetings
Reading messages... (press Ctrl-C to quit)
1) "subscribe"
2) "greetings"
3) (integer) 1
```

(2) Subscribe to a message channel pattern.

```redis
127.0.0.1:6379> psubscribe greet*
Reading messages... (press Ctrl-C to quit)
1) "psubscribe"
2) "greet*"
3) (integer) 1
```

(3) Publish a message to a message channel.

```
127.0.0.1:6379> publish greetings "hello redis"
(integer) 1
```

## 2.8 Transactions

(1) Redis has limited support for transactions.

(2) Group multiple Redis commands together and execute them as a single unit.

(3) Redis transactions are atomic.

(4) No rollback: if one command in a transaction fails during the execution of the transaction, the rest of the commands will still be executed.

(5) But if an error occurs when you are queueing up the commands via multi for the transaction, then the transaction won't run.

### 2.8.1 Using transactions

Launch two Redis clients RC1 and RC2. In RC1, we use Redis transactions to simulate a transfer from account a to account b. In RC2, we initialize the account balances and try interfering with the transfer.

(1) Initialize the account balances.

```redis
## RC2
127.0.0.1:6379> set account-a 100
OK
127.0.0.1:6379> set account-b 200
OK
```

(2) Start the transfer.

```redis
## RC1
127.0.0.1:6379> multi
OK
127.0.0.1:6379> incrby account-a -50
QUEUED
127.0.0.1:6379> incrby account-b 50
QUEUED
```

Note that the transaction isn't exeucted at this time.

(3) Check the account balances and verify that the transfer hasn't occurred. Also deposit 300 into account a.

``` redis
## RC2
127.0.0.1:6379> get account-a
"100"
127.0.0.1:6379> get account-b
"200"
127.0.0.1:6379> incrby account-a 300
(integer) 400
```

(4) Execute the transfer transaction.

```redis
## RC1
127.0.0.1:6379> exec
1) (integer) 350
2) (integer) 250
```

(5) Start another transfer but put a watch on acount a's balance.

```
## RC1
127.0.0.1:6379> watch account-a
OK
127.0.0.1:6379> multi
OK
127.0.0.1:6379> incrby account-a -50
QUEUED
127.0.0.1:6379> incrby account-b +50
QUEUED
```

(6) Modify the account a's balance during the transfer.

```
## RC2
127.0.0.1:6379> set account-a 0
OK
127.0.0.1:6379> set account-a 25
OK
```

(7) Execute the transfer transaction, but the transaction will NOT be executed due to the modification in (6).

```
## RC1
127.0.0.1:6379> exec
(nil)
```